In [17]:
import numpy as np
import torch
import torch.nn as nn
from training import train
from NP import NP
import matplotlib.pyplot as plt
import random
from PIL import Image
from training import train
from NP import NP
import torchvision
import torchvision.datasets as datasets

#imports MNIST training and testing data
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [18]:
def img_to_vector(imgs):
    data = [torch.from_numpy(np.asarray(img, dtype="int32")).unsqueeze(0) for img in imgs]
    return torch.cat(data, dim=0).float()

def vector_to_image(vec):
    array = vec.reshape(28, 28).detach().numpy()
    img = Image.fromarray(array)
    return img.convert('RGB')

mnist_trainset_images = [mnist_trainset[i][0] for i in range(len(mnist_trainset))]
mnist_trainset_digits = [mnist_trainset[i][1] for i in range(len(mnist_trainset))]

mnist_trainset_arrays = img_to_vector(mnist_trainset_images)
mnist_trainset_digits = torch.LongTensor(mnist_trainset_digits).view(-1, 1)

In [19]:
#batches data for training
def batch(data, batched_size):
    batched = [data[k*batched_size:(k+1)*batched_size] for k in range(len(data)//batched_size)]
    return batched

batched_arrays = batch(mnist_trainset_arrays, 32)
batched_digits = batch(mnist_trainset_digits, 32)
onehot = torch.zeros(len(mnist_trainset_arrays), 10)
onehot = list(onehot.scatter(-1, mnist_trainset_digits, 1))
batched_onehot = batch(onehot, 32)

data = [[batched_arrays[i], batched_digits[i]] for i in range(len(batched_onehot))]

mnist_testset_images = [mnist_testset[i][0] for i in range(len(mnist_testset))]
mnist_testset_digits = [mnist_testset[i][1] for i in range(len(mnist_testset))]

mnist_testset_arrays = img_to_vector(mnist_testset_images)
mnist_testset_digits = torch.LongTensor(mnist_testset_digits).view(-1, 1)

In [81]:
class CNN(torch.nn.Module):
    #input size ([batch_size, 1, 28, 28])
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(196, 64)
        self.fc2 = torch.nn.Linear(64, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        l1 = self.relu(self.conv1(x))
        l2 = self.pool(l1)
        
        l2 = l2.view(-1, 196)
        l3 = self.relu(self.fc1(l2))
        l4 = self.fc2(l3)
        if cnn.training:
            return l4
        else:
            m = l4.max()
            num = list(l4.detach()[0]).index(m)
            return num

In [82]:
cnn = CNN()
lossf = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=3e-3)

In [98]:
#trains the CNN. After every epoch a random testset image is chosen and classified.
def train(cnn, data, batch_size, epochs):
    cnn.train()
    for epoch in range(epochs):
        batch = 0
        epoch_loss = 0
        for point, onehot in data:
            
            optimizer.zero_grad()
            outputs = cnn(point.reshape(batch_size, 1, 28, 28))
            
            loss = lossf(outputs, onehot.reshape(batch_size))
            loss.backward()
            optimizer.step()
            batch += 1
            epoch_loss += loss
                
        print('EPOCH {} LOSS {}'.format(epoch, epoch_loss))
        i = np.random.randint(len(mnist_testset_images))
        display(mnist_testset_images[i])
        
        cnn.training = False
        print(cnn(mnist_testset_arrays[i].unsqueeze(0).unsqueeze(0)))
        cnn.train()

In [99]:
train(cnn, data, 32, 25)

EPOCH 0 LOSS 0.19450101256370544


6
EPOCH 1 LOSS 0.07174747437238693


4
EPOCH 2 LOSS 0.03926028311252594


1
EPOCH 3 LOSS 0.023075692355632782


8
EPOCH 4 LOSS 0.015286512672901154


...

8
EPOCH 10 LOSS 0.005123339593410492


1
EPOCH 11 LOSS 0.004611030220985413


9
EPOCH 12 LOSS 0.004187248647212982


2


KeyboardInterrupt: 

2